# Quality Control Command Center (QC) — Project

## Step 1: Foundation (Data Understanding & KPI Definitions)

### Goal
Define what our QC system should monitor daily, and establish consistent KPI definitions that will be used throughout the project (detection → RCA → validation → dashboard).

---

## 1.1 Dataset Overview (Files)
This project uses a synthetic but realistic QC dataset covering:
- Supplier quality (supplier baseline risk)
- Dimensional measurements (OOS / drift signals)
- Defect events (quality issues + costs)
- Engineering changes (EO / model change windows)
- Actions (containment / corrective / validation tracking)

**Input CSV files**
- `suppliers.csv`
- `parts_master.csv`
- `engineering_changes.csv`
- `eo_impacted_parts.csv`
- `production_lots.csv`
- `dim_measurements.csv`
- `defects.csv`
- `actions.csv`

---

## 1.2 Key Keys & Relationships (How tables connect)
We will treat `lot_id` as the core operational join key.

**Primary join keys**
- `lot_id`: connects production lots ↔ measurements ↔ defects
- `part_id`: connects lots/measurements/defects ↔ parts metadata
- `supplier_id`: connects lots/measurements/defects ↔ supplier metadata
- `eo_id`: connects measurements/defects ↔ engineering changes (EO)
- `measure_id`: connects dimensional defects ↔ a specific measurement record

**Important mappings**
- `eo_impacted_parts.csv` provides EO → impacted `part_id` mapping (many-to-many).

---

## 1.3 KPI Definitions (Daily Monitoring)
We will compute KPIs at multiple granularities (daily, supplier, part, line, shift).  
At minimum, our daily QC monitoring will track:

| KPI | Definition | Source Tables |
|---|---|---|
| Defect Rate (per 1,000) | `defect_count / qty_used * 1000` | production_lots + defects |
| PPM | `defect_count / qty_used * 1,000,000` | production_lots + defects |
| OOS Rate | `(# measurements out of spec) / (total measurements)` | dim_measurements |
| Cost | `sum(scrap_cost_usd + rework_cost_usd)` | defects |
| Risk Score | Composite score based on OOS, severity, cost, EO activity | defects + dim_measurements + engineering_changes |

---

## 1.4 Risk Score (Initial v1 specification)
We need a simple, explainable score that prioritizes investigation.
A practical first version is:

- Base components:
  - OOS Rate (higher → higher risk)
  - Severity mix (A/B/C)
  - Cost (scrap + rework)
  - EO activity flag (EO active → additional risk)

We will implement v1 as:
- `severity_points = 3*count(A) + 2*count(B) + 1*count(C)`
- `risk_score = 100*(oos_rate) + 0.02*(total_cost) + 2*(severity_points) + 5*(eo_active_flag)`

This is intentionally simple for Step 1.  
We can calibrate weights later based on what makes the dashboard most actionable.

---

## 1.5 Deliverables for Step 1
By the end of Step 1, we will produce:
1) A KPI definition table (above) that becomes our project standard
2) A data dictionary (columns + types + missingness + descriptions)
3) Basic relationship checks (key uniqueness + referential integrity)


In [1]:
# ============================================================
# Step 1 — Foundation: Data Loading, Setup, and Documentation
# ============================================================

import os
import pandas as pd
import numpy as np

# ----------------------------
# 0) Path configuration
# ----------------------------
DATA_DIR = "data"  # assumes the notebook is run from the project root
assert os.path.isdir(DATA_DIR), f"DATA_DIR not found: {os.path.abspath(DATA_DIR)}"

FILES = {
    "suppliers": "suppliers.csv",
    "parts": "parts_master.csv",
    "engineering_changes": "engineering_changes.csv",
    "eo_impacted_parts": "eo_impacted_parts.csv",
    "production_lots": "production_lots.csv",
    "dim_measurements": "dim_measurements.csv",
    "defects": "defects.csv",
    "actions": "actions.csv",
}

def load_csv(name: str) -> pd.DataFrame:
    """Load a CSV from DATA_DIR with a consistent setup."""
    path = os.path.join(DATA_DIR, FILES[name])
    df = pd.read_csv(path)
    return df

# ----------------------------
# 1) Load all tables
# ----------------------------
suppliers = load_csv("suppliers")
parts = load_csv("parts")
engineering_changes = load_csv("engineering_changes")
eo_impacted_parts = load_csv("eo_impacted_parts")
production_lots = load_csv("production_lots")
dim_measurements = load_csv("dim_measurements")
defects = load_csv("defects")
actions = load_csv("actions")

tables = {
    "suppliers": suppliers,
    "parts": parts,
    "engineering_changes": engineering_changes,
    "eo_impacted_parts": eo_impacted_parts,
    "production_lots": production_lots,
    "dim_measurements": dim_measurements,
    "defects": defects,
    "actions": actions,
}

# ----------------------------
# 2) Basic type casting (dates/timestamps)
# ----------------------------
# Convert date-like columns to datetime for consistent time-based analyses later.
production_lots["date"] = pd.to_datetime(production_lots["date"])
defects["date"] = pd.to_datetime(defects["date"])
actions["start_date"] = pd.to_datetime(actions["start_date"], errors="coerce")

engineering_changes["start_date"] = pd.to_datetime(engineering_changes["start_date"])
engineering_changes["end_date"] = pd.to_datetime(engineering_changes["end_date"])

dim_measurements["timestamp"] = pd.to_datetime(dim_measurements["timestamp"])

# Ensure EO id is treated consistently (string) even if missing.
dim_measurements["eo_active"] = dim_measurements["eo_active"].astype("string")
defects["eo_id"] = defects["eo_id"].astype("string")

# ----------------------------
# 3) Quick sanity checks (shape + head)
# ----------------------------
for name, df in tables.items():
    print(f"{name:>20s} | rows={len(df):>10,d} | cols={df.shape[1]:>3d}")

display(production_lots.head(3))
display(dim_measurements.head(3))
display(defects.head(3))

# ----------------------------
# 4) Relationship checks (keys and referential integrity)
# ----------------------------
def check_unique(df: pd.DataFrame, key: str, table_name: str) -> None:
    """Check if a key column is unique (useful for primary key validation)."""
    n = len(df)
    nunique = df[key].nunique(dropna=False)
    if nunique != n:
        print(f"[WARN] {table_name}.{key} not unique: rows={n}, unique={nunique}")
    else:
        print(f"[OK]   {table_name}.{key} is unique")

def check_fk(child: pd.DataFrame, child_key: str, parent: pd.DataFrame, parent_key: str, label: str) -> None:
    """Check foreign-key-like coverage: child keys should exist in parent keys."""
    child_vals = set(child[child_key].dropna().astype(str).unique())
    parent_vals = set(parent[parent_key].dropna().astype(str).unique())
    missing = child_vals - parent_vals
    if len(missing) > 0:
        print(f"[WARN] FK check failed ({label}): missing={len(missing)} (showing up to 5): {list(missing)[:5]}")
    else:
        print(f"[OK]   FK check passed ({label})")

# Primary key candidates
check_unique(suppliers, "supplier_id", "suppliers")
check_unique(parts, "part_id", "parts")
check_unique(production_lots, "lot_id", "production_lots")

# Foreign key-like checks
check_fk(parts, "supplier_id", suppliers, "supplier_id", "parts.supplier_id -> suppliers.supplier_id")
check_fk(production_lots, "part_id", parts, "part_id", "production_lots.part_id -> parts.part_id")
check_fk(production_lots, "supplier_id", suppliers, "supplier_id", "production_lots.supplier_id -> suppliers.supplier_id")
check_fk(dim_measurements, "lot_id", production_lots, "lot_id", "dim_measurements.lot_id -> production_lots.lot_id")
check_fk(defects, "lot_id", production_lots, "lot_id", "defects.lot_id -> production_lots.lot_id")
check_fk(defects, "part_id", parts, "part_id", "defects.part_id -> parts.part_id")
check_fk(defects, "supplier_id", suppliers, "supplier_id", "defects.supplier_id -> suppliers.supplier_id")

# Optional link: dimensional defects -> specific measurement id
if "linked_measure_id" in defects.columns:
    meas_ids = set(dim_measurements["measure_id"].astype(str).unique())
    linked = defects["linked_measure_id"].dropna().astype(str)
    missing_links = set(linked.unique()) - meas_ids
    if len(missing_links) > 0:
        print(f"[WARN] defects.linked_measure_id has missing measure_id links: {len(missing_links)} (up to 5): {list(missing_links)[:5]}")
    else:
        print("[OK]   defects.linked_measure_id links are valid (when present)")

# ----------------------------
# 5) Create a Data Dictionary (columns, dtype, missingness)
# ----------------------------
def build_data_dictionary(tables: dict) -> pd.DataFrame:
    """
    Build a data dictionary with:
    - table name
    - column name
    - dtype
    - missing count / missing rate
    - example values
    This is a mechanical dictionary; we will add human descriptions manually as needed.
    """
    rows = []
    for tname, df in tables.items():
        for col in df.columns:
            missing_count = int(df[col].isna().sum())
            missing_rate = float(missing_count / len(df)) if len(df) > 0 else 0.0
            # sample up to 3 non-null examples
            examples = df[col].dropna().astype(str).head(3).tolist()
            rows.append({
                "table": tname,
                "column": col,
                "dtype": str(df[col].dtype),
                "missing_count": missing_count,
                "missing_rate": round(missing_rate, 4),
                "example_values": ", ".join(examples)
            })
    return pd.DataFrame(rows)

data_dictionary = build_data_dictionary(tables)

# Save the data dictionary as a CSV for documentation.
dict_path = os.path.join(DATA_DIR, "data_dictionary_step1.csv")
data_dictionary.to_csv(dict_path, index=False)
print(f"✅ Data dictionary saved: {os.path.abspath(dict_path)}")

display(data_dictionary.head(20))

# ----------------------------
# 6) KPI Definitions (as a structured table, for documentation)
# ----------------------------
kpi_definitions = pd.DataFrame([
    {
        "KPI": "Defect Rate (per 1,000)",
        "Formula": "defect_count / qty_used * 1000",
        "Interpretation": "How many defects occur per 1,000 used units (lot-level or aggregated).",
        "Tables": "production_lots + defects"
    },
    {
        "KPI": "PPM",
        "Formula": "defect_count / qty_used * 1,000,000",
        "Interpretation": "Defects per million units used (standard supplier quality metric).",
        "Tables": "production_lots + defects"
    },
    {
        "KPI": "OOS Rate",
        "Formula": "out_of_spec_measurements / total_measurements",
        "Interpretation": "Fraction of measurements outside spec limits (LSL/USL).",
        "Tables": "dim_measurements"
    },
    {
        "KPI": "Cost",
        "Formula": "sum(scrap_cost_usd + rework_cost_usd)",
        "Interpretation": "Total quality cost captured from scrap + rework.",
        "Tables": "defects"
    },
    {
        "KPI": "Risk Score (v1)",
        "Formula": "100*oos_rate + 0.02*total_cost + 2*severity_points + 5*eo_active_flag",
        "Interpretation": "Explainable composite prioritization score for investigation (weights adjustable).",
        "Tables": "defects + dim_measurements + engineering_changes"
    }
])

kpi_path = os.path.join(DATA_DIR, "kpi_definitions_step1.csv")
kpi_definitions.to_csv(kpi_path, index=False)
print(f"✅ KPI definitions saved: {os.path.abspath(kpi_path)}")

display(kpi_definitions)

# ----------------------------
# 7) (Optional) Severity points helper table (for later steps)
# ----------------------------
severity_points_map = {"A": 3, "B": 2, "C": 1}
print("Severity points mapping:", severity_points_map)


           suppliers | rows=        12 | cols=  6
               parts | rows=        36 | cols=  9
 engineering_changes | rows=         6 | cols=  6
   eo_impacted_parts | rows=        35 | cols=  2
     production_lots | rows=     4,901 | cols=  8
    dim_measurements | rows=    59,526 | cols= 17
             defects | rows=       619 | cols= 15
             actions | rows=       131 | cols=  8


,lot_id,date,plant_line,shift,part_id,supplier_id,qty_received,qty_used
0,L0000001,2025-07-01,ChassisLine-1,B,P0013,S007,309,224
1,L0000002,2025-07-01,Final-1,B,P0030,S009,258,214
2,L0000003,2025-07-01,BodyLine-1,A,P0010,S005,366,321


,measure_id,timestamp,lot_id,part_id,supplier_id,plant_line,shift,dimension_name,value_mm,nominal,lsl,usl,is_out_of_spec,deviation,tool_id,operator_id,eo_active
0,M000000001,2025-07-01 16:33:00,L0000001,P0013,S007,ChassisLine-1,B,Gap_mm,39.8354,39.898,38.898,40.897,0,-0.0626,T05,O004,<NA>
1,M000000002,2025-07-01 19:34:00,L0000001,P0013,S007,ChassisLine-1,B,Gap_mm,39.9436,39.898,38.898,40.897,0,0.0456,T05,O004,<NA>
2,M000000003,2025-07-01 10:16:00,L0000001,P0013,S007,ChassisLine-1,B,Gap_mm,39.7673,39.898,38.898,40.897,0,-0.1307,T05,O004,<NA>


,defect_id,date,lot_id,part_id,supplier_id,plant_line,shift,station,defect_type,severity,stage,eo_id,linked_measure_id,scrap_cost_usd,rework_cost_usd
0,D000000001,2025-07-01,L0000013,P0027,S005,Final-1,A,FinalInspect,Dimensional,B,MassProduction,<NA>,M000000162,17.60,6.68
1,D000000002,2025-07-01,L0000014,P0025,S010,Final-1,B,FinalInspect,Dimensional,A,MassProduction,<NA>,M000000171,115.78,60.84
2,D000000003,2025-07-01,L0000021,P0005,S001,BodyLine-1,A,Paint,Dimensional,B,Pilot,<NA>,M000000237,187.84,101.26


[OK]   suppliers.supplier_id is unique
[OK]   parts.part_id is unique
[OK]   production_lots.lot_id is unique
[OK]   FK check passed (parts.supplier_id -> suppliers.supplier_id)
[OK]   FK check passed (production_lots.part_id -> parts.part_id)
[OK]   FK check passed (production_lots.supplier_id -> suppliers.supplier_id)
[OK]   FK check passed (dim_measurements.lot_id -> production_lots.lot_id)
[OK]   FK check passed (defects.lot_id -> production_lots.lot_id)
[OK]   FK check passed (defects.part_id -> parts.part_id)
[OK]   FK check passed (defects.supplier_id -> suppliers.supplier_id)
[OK]   defects.linked_measure_id links are valid (when present)
✅ Data dictionary saved: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\data_dictionary_step1.csv


,table,column,dtype,missing_count,missing_rate,example_values
0,suppliers,supplier_id,object,0,0.0,"S001, S002, S003"
1,suppliers,supplier_name,object,0,0.0,"Supplier_1, Supplier_2, Supplier_3"
2,suppliers,location,object,0,0.0,"MI, TN, KY"
3,suppliers,quality_rating,object,0,0.0,"A, B, C"
4,suppliers,lead_time_days,int64,0,0.0,"11, 4, 10"
5,suppliers,defect_ppm_baseline,float64,0,0.0,"194.09, 287.0, 620.5"
6,parts,part_id,object,0,0.0,"P0001, P0002, P0003"
7,parts,part_name,object,0,0.0,"Electrical_Part_1, Electrical_Part_2, Body_Part_3"
8,parts,system,object,0,0.0,"Electrical, Electrical, Body"
9,parts,spec_dimension,object,0,0.0,"PinDepth_mm, ClipLength_mm, Gap_mm"


✅ KPI definitions saved: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\kpi_definitions_step1.csv


,KPI,Formula,Interpretation,Tables
0,"Defect Rate (per 1,000)",defect_count / qty_used * 1000,"How many defects occur per 1,000 used units (l...",production_lots + defects
1,PPM,"defect_count / qty_used * 1,000,000",Defects per million units used (standard suppl...,production_lots + defects
2,OOS Rate,out_of_spec_measurements / total_measurements,Fraction of measurements outside spec limits (...,dim_measurements
3,Cost,sum(scrap_cost_usd + rework_cost_usd),Total quality cost captured from scrap + rework.,defects
4,Risk Score (v1),100*oos_rate + 0.02*total_cost + 2*severity_po...,Explainable composite prioritization score for...,defects + dim_measurements + engineering_changes


Severity points mapping: {'A': 3, 'B': 2, 'C': 1}


## Step 2: Detection (Quality Issue Monitoring & Alerts)

### Goal
Automatically detect emerging quality issues using daily monitoring signals:
- Defect spikes (rate-based anomalies)
- OOS spikes (dimensional out-of-spec anomalies)
- Cost spikes (scrap + rework anomalies)
- EO-related risk (issues during active engineering changes)

This step produces:
1) A daily KPI aggregation table (`daily_kpi.csv`)
2) An alert table with prioritized events (`alerts_step2.csv`)

---

### Detection Logic (v1)
We implement simple, explainable rules for anomaly detection:

**A) Defect Spike Alert**
- Compare the most recent 7-day defect rate to a trailing 30-day baseline.
- Trigger if `rate_7d >= 1.8 * rate_30d` AND the absolute rate exceeds a minimum threshold.

**B) OOS Rate Alert**
- Trigger if daily OOS rate exceeds a threshold (e.g., 8%) and minimum measurement volume is met.

**C) Cost Spike Alert**
- Trigger if recent 7-day cost per 1,000 units is significantly higher than trailing baseline.

**D) EO Risk Flag**
- If defects or measurements are associated with an active EO window, add risk weight.

These rules are intentionally simple in Step 2.
We can replace them later with statistical control charts (SPC) or ML-based anomaly detection.


In [2]:
# ============================================================
# Step 2 — Detection: Build daily KPI table + generate alerts
# All comments and explanations are written in English.
# ============================================================

DATA_DIR = "data"

# ----------------------------
# 1) Load required tables
# ----------------------------
production_lots = pd.read_csv(os.path.join(DATA_DIR, "production_lots.csv"))
dim_measurements = pd.read_csv(os.path.join(DATA_DIR, "dim_measurements.csv"))
defects = pd.read_csv(os.path.join(DATA_DIR, "defects.csv"))
engineering_changes = pd.read_csv(os.path.join(DATA_DIR, "engineering_changes.csv"))

# ----------------------------
# 2) Parse dates/timestamps consistently
# ----------------------------
production_lots["date"] = pd.to_datetime(production_lots["date"])
dim_measurements["timestamp"] = pd.to_datetime(dim_measurements["timestamp"])
defects["date"] = pd.to_datetime(defects["date"])

engineering_changes["start_date"] = pd.to_datetime(engineering_changes["start_date"])
engineering_changes["end_date"] = pd.to_datetime(engineering_changes["end_date"])

# Normalize EO id columns to string for stable joins/flags
dim_measurements["eo_active"] = dim_measurements["eo_active"].astype("string")
defects["eo_id"] = defects["eo_id"].astype("string")

# ----------------------------
# 3) Build daily measurement KPIs (OOS metrics)
#    Grain: date x supplier_id x part_id x plant_line x shift
# ----------------------------
dim_measurements["date"] = dim_measurements["timestamp"].dt.floor("D")

meas_daily = (
    dim_measurements
    .groupby(["date", "supplier_id", "part_id", "plant_line", "shift"], as_index=False)
    .agg(
        meas_n=("measure_id", "count"),
        oos_n=("is_out_of_spec", "sum"),
        oos_rate=("is_out_of_spec", "mean"),
        avg_deviation=("deviation", "mean"),
        abs_avg_deviation=("deviation", lambda s: np.mean(np.abs(s))),
        eo_active_flag=("eo_active", lambda s: int(s.notna().any()))
    )
)

# ----------------------------
# 4) Build daily defect KPIs (defect count, severity, costs)
#    Grain: date x supplier_id x part_id x plant_line x shift
# ----------------------------
# Create severity point mapping for an explainable risk score.
severity_points_map = {"A": 3, "B": 2, "C": 1}
defects["severity_points"] = defects["severity"].map(severity_points_map).fillna(0).astype(int)
defects["total_cost_usd"] = defects["scrap_cost_usd"].fillna(0) + defects["rework_cost_usd"].fillna(0)

def_daily = (
    defects
    .groupby(["date", "supplier_id", "part_id", "plant_line", "shift"], as_index=False)
    .agg(
        defect_n=("defect_id", "count"),
        severity_points=("severity_points", "sum"),
        cost_usd=("total_cost_usd", "sum"),
        eo_active_flag_def=("eo_id", lambda s: int(s.notna().any()))
    )
)

# ----------------------------
# 5) Build daily lot usage (denominator for rates)
#    Grain: date x supplier_id x part_id x plant_line x shift
# ----------------------------
lots_daily = (
    production_lots
    .groupby(["date", "supplier_id", "part_id", "plant_line", "shift"], as_index=False)
    .agg(
        qty_used=("qty_used", "sum"),
        qty_received=("qty_received", "sum"),
        lot_n=("lot_id", "nunique")
    )
)

# ----------------------------
# 6) Combine into a single daily KPI table
# ----------------------------
daily_kpi = lots_daily.merge(def_daily, on=["date", "supplier_id", "part_id", "plant_line", "shift"], how="left")
daily_kpi = daily_kpi.merge(meas_daily, on=["date", "supplier_id", "part_id", "plant_line", "shift"], how="left")

# Fill missing metrics (e.g., days with no defects or no measurements)
for col in ["defect_n", "severity_points", "cost_usd", "eo_active_flag_def"]:
    if col in daily_kpi.columns:
        daily_kpi[col] = daily_kpi[col].fillna(0)

for col in ["meas_n", "oos_n", "oos_rate", "avg_deviation", "abs_avg_deviation", "eo_active_flag"]:
    if col in daily_kpi.columns:
        daily_kpi[col] = daily_kpi[col].fillna(0)

# Compute defect rates
# - Defect Rate per 1,000 units: defects / qty_used * 1000
# - PPM: defects / qty_used * 1,000,000
daily_kpi["defect_rate_per_1000"] = np.where(
    daily_kpi["qty_used"] > 0,
    daily_kpi["defect_n"] / daily_kpi["qty_used"] * 1000,
    0.0
)

daily_kpi["ppm"] = np.where(
    daily_kpi["qty_used"] > 0,
    daily_kpi["defect_n"] / daily_kpi["qty_used"] * 1_000_000,
    0.0
)

# Cost normalization (cost per 1,000 units used)
daily_kpi["cost_per_1000_units"] = np.where(
    daily_kpi["qty_used"] > 0,
    daily_kpi["cost_usd"] / daily_kpi["qty_used"] * 1000,
    0.0
)

# EO flag: if either defects or measurements have EO activity, set 1
daily_kpi["eo_flag"] = ((daily_kpi["eo_active_flag_def"] > 0) | (daily_kpi["eo_active_flag"] > 0)).astype(int)

# Risk score v1 (simple explainable formula; weights can be tuned later)
# risk_score = 100*oos_rate + 0.02*cost_usd + 2*severity_points + 5*eo_flag
daily_kpi["risk_score_v1"] = (
    100 * daily_kpi["oos_rate"]
    + 0.02 * daily_kpi["cost_usd"]
    + 2 * daily_kpi["severity_points"]
    + 5 * daily_kpi["eo_flag"]
)

# Save daily KPI table
daily_kpi_path = os.path.join(DATA_DIR, "daily_kpi.csv")
daily_kpi.to_csv(daily_kpi_path, index=False)
print(f"✅ Saved daily KPI table: {os.path.abspath(daily_kpi_path)}")
print("daily_kpi shape:", daily_kpi.shape)

# ----------------------------
# 7) Detection rules (v1) — build alerts
# ----------------------------
# Rule configuration (tuneable)
OOS_RATE_THRESHOLD = 0.08         # 8%
MIN_MEAS_FOR_OOS_ALERT = 25       # require enough measurements
MIN_QTY_FOR_RATE_ALERT = 150      # require enough volume for stable rate
SPIKE_RATIO_THRESHOLD = 1.8       # 7-day vs 30-day baseline ratio
MIN_DEFECT_RATE_PER_1000 = 0.8    # minimum absolute defect rate to avoid tiny-noise alerts
MIN_COST_PER_1000 = 25.0          # minimum absolute cost to avoid tiny-noise alerts

# Helper: compute rolling baselines within each (supplier, part, line, shift) group
group_cols = ["supplier_id", "part_id", "plant_line", "shift"]
daily_kpi_sorted = daily_kpi.sort_values(["date"] + group_cols).copy()

def add_rolling_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add rolling 7-day and 30-day features for anomaly detection."""
    df = df.sort_values("date").copy()

    # Rolling means for rate and cost (past windows including current day)
    df["defect_rate_7d"] = df["defect_rate_per_1000"].rolling(window=7, min_periods=4).mean()
    df["defect_rate_30d"] = df["defect_rate_per_1000"].rolling(window=30, min_periods=10).mean()

    df["cost_7d"] = df["cost_per_1000_units"].rolling(window=7, min_periods=4).mean()
    df["cost_30d"] = df["cost_per_1000_units"].rolling(window=30, min_periods=10).mean()

    # Avoid division by zero
    df["defect_spike_ratio"] = np.where(df["defect_rate_30d"] > 0, df["defect_rate_7d"] / df["defect_rate_30d"], np.nan)
    df["cost_spike_ratio"] = np.where(df["cost_30d"] > 0, df["cost_7d"] / df["cost_30d"], np.nan)

    return df

daily_kpi_feat = (
    daily_kpi_sorted
    .groupby(group_cols, group_keys=False)
    .apply(add_rolling_features)
)

# ----------------------------
# 8) Generate alert flags
# ----------------------------
alerts = []

# A) Defect spike alerts
mask_def_spike = (
    (daily_kpi_feat["qty_used"] >= MIN_QTY_FOR_RATE_ALERT) &
    (daily_kpi_feat["defect_rate_per_1000"] >= MIN_DEFECT_RATE_PER_1000) &
    (daily_kpi_feat["defect_spike_ratio"] >= SPIKE_RATIO_THRESHOLD)
)

def_spike_df = daily_kpi_feat.loc[mask_def_spike, :].copy()
def_spike_df["alert_type"] = "DEFECT_SPIKE"
def_spike_df["alert_score"] = (
    10 * def_spike_df["defect_spike_ratio"].fillna(0)
    + 0.5 * def_spike_df["risk_score_v1"]
)
alerts.append(def_spike_df)

# B) OOS rate alerts
mask_oos = (
    (daily_kpi_feat["meas_n"] >= MIN_MEAS_FOR_OOS_ALERT) &
    (daily_kpi_feat["oos_rate"] >= OOS_RATE_THRESHOLD)
)

oos_df = daily_kpi_feat.loc[mask_oos, :].copy()
oos_df["alert_type"] = "OOS_HIGH"
oos_df["alert_score"] = (
    500 * oos_df["oos_rate"]  # strongly prioritize high OOS
    + 0.3 * oos_df["risk_score_v1"]
)
alerts.append(oos_df)

# C) Cost spike alerts
mask_cost_spike = (
    (daily_kpi_feat["qty_used"] >= MIN_QTY_FOR_RATE_ALERT) &
    (daily_kpi_feat["cost_per_1000_units"] >= MIN_COST_PER_1000) &
    (daily_kpi_feat["cost_spike_ratio"] >= SPIKE_RATIO_THRESHOLD)
)

cost_df = daily_kpi_feat.loc[mask_cost_spike, :].copy()
cost_df["alert_type"] = "COST_SPIKE"
cost_df["alert_score"] = (
    8 * cost_df["cost_spike_ratio"].fillna(0)
    + 0.4 * cost_df["risk_score_v1"]
)
alerts.append(cost_df)

# Combine alerts into one table
if len(alerts) > 0:
    alerts_df = pd.concat(alerts, ignore_index=True)
else:
    alerts_df = pd.DataFrame(columns=list(daily_kpi_feat.columns) + ["alert_type", "alert_score"])

# Keep a concise set of columns for alert review
alert_cols = [
    "date", "supplier_id", "part_id", "plant_line", "shift",
    "alert_type", "alert_score",
    "qty_used", "defect_n", "defect_rate_per_1000", "ppm",
    "meas_n", "oos_rate",
    "cost_usd", "cost_per_1000_units",
    "severity_points", "eo_flag",
    "risk_score_v1",
    "defect_rate_7d", "defect_rate_30d", "defect_spike_ratio",
    "cost_7d", "cost_30d", "cost_spike_ratio",
]

# Some rolling columns might be missing if no enough periods; keep intersection only
alert_cols = [c for c in alert_cols if c in alerts_df.columns]

alerts_df = alerts_df[alert_cols].copy()

# Sort by alert score (descending) to show priorities
alerts_df = alerts_df.sort_values(["date", "alert_score"], ascending=[True, False])

alerts_path = os.path.join(DATA_DIR, "alerts_step2.csv")
alerts_df.to_csv(alerts_path, index=False)
print(f"✅ Saved alerts table: {os.path.abspath(alerts_path)}")
print("alerts shape:", alerts_df.shape)

# Show the most recent 20 alerts as a quick check
latest_date = alerts_df["date"].max() if len(alerts_df) else None
print("Latest alert date:", latest_date)

display(alerts_df.tail(20))


✅ Saved daily KPI table: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\daily_kpi.csv
daily_kpi shape: (4901, 23)
✅ Saved alerts table: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\alerts_step2.csv
alerts shape: (225, 24)
Latest alert date: 2026-01-30 00:00:00


C:\Users\Hyunwoo Jang\AppData\Local\Temp\ipykernel_29092\795801161.py:173: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(add_rolling_features)


,date,supplier_id,part_id,plant_line,shift,alert_type,alert_score,qty_used,defect_n,defect_rate_per_1000,...,cost_per_1000_units,severity_points,eo_flag,risk_score_v1,defect_rate_7d,defect_rate_30d,defect_spike_ratio,cost_7d,cost_30d,cost_spike_ratio
99,2026-01-15,S011,P0011,Final-1,A,DEFECT_SPIKE,32.878071,157,1.0,6.369427,...,286.305732,1.0,0,2.8990,0.909918,0.289519,3.142857,40.900819,13.013897,3.142857
215,2026-01-15,S011,P0011,Final-1,A,COST_SPIKE,26.302457,157,1.0,6.369427,...,286.305732,1.0,0,2.8990,0.909918,0.289519,3.142857,40.900819,13.013897,3.142857
100,2026-01-16,S008,P0029,Final-1,A,DEFECT_SPIKE,45.889143,297,1.0,3.367003,...,347.474747,2.0,0,6.0640,0.481000,0.112233,4.285714,49.639250,11.582492,4.285714
216,2026-01-16,S008,P0029,Final-1,A,COST_SPIKE,36.711314,297,1.0,3.367003,...,347.474747,2.0,0,6.0640,0.481000,0.112233,4.285714,49.639250,11.582492,4.285714
101,2026-01-18,S012,P0026,BodyLine-1,B,DEFECT_SPIKE,30.049057,267,1.0,3.745318,...,713.932584,1.0,0,5.8124,0.535045,0.197122,2.714286,101.990369,37.575399,2.714286
217,2026-01-18,S012,P0026,BodyLine-1,B,COST_SPIKE,24.039246,267,1.0,3.745318,...,713.932584,1.0,0,5.8124,0.535045,0.197122,2.714286,101.990369,37.575399,2.714286
102,2026-01-20,S006,P0019,BodyLine-1,A,DEFECT_SPIKE,23.688516,259,1.0,3.861004,...,329.768340,1.0,0,3.7082,0.551572,0.252616,2.183442,47.109763,22.315698,2.111059
218,2026-01-20,S006,P0019,BodyLine-1,A,COST_SPIKE,18.371755,259,1.0,3.861004,...,329.768340,1.0,0,3.7082,0.551572,0.252616,2.183442,47.109763,22.315698,2.111059
103,2026-01-21,S007,P0007,BodyLine-2,A,DEFECT_SPIKE,21.950494,178,1.0,5.617978,...,564.775281,2.0,0,6.0106,0.802568,0.423626,1.894519,80.682183,64.655940,1.247870
104,2026-01-24,S002,P0024,TrimLine-1,B,DEFECT_SPIKE,26.009214,151,1.0,6.622517,...,479.139073,1.0,0,3.4470,2.356406,0.970285,2.428571,141.455863,58.246532,2.428571


## Step 3: Root Cause Analysis (RCA)

### Goal
Given detected quality alerts (Step 2), identify the most likely root-cause drivers by quantifying evidence across:
- Supplier effects (supplier-specific risk and drift)
- Part effects (CTQ, dimensional sensitivity)
- Line/Shift effects (manufacturing conditions)
- Engineering changes (EO active windows)
- Measurement-to-defect linkage (OOS → defects relationship)

This step produces:
1) `rca_cases_step3.csv`: a structured RCA summary for the top alerts
2) `rca_evidence_step3.csv`: supporting evidence metrics for each RCA case

---

### RCA Approach (Explainable v1)
For each alert, we create an investigation “case” at the same grain:
(date, supplier_id, part_id, plant_line, shift)

We then compute evidence signals:
1) **Comparison vs baseline**  
   - Compare recent window (e.g., last 7 days) vs historical baseline (e.g., prior 30 days)

2) **Driver ranking**  
   - Within the same day (or week), rank how extreme the supplier/part/line/shift is
   - Identify “top contributors” to defect_n, cost_usd, OOS rate

3) **Causal hints (not proof)**  
   - EO flag presence increases likelihood of change-driven issues
   - Strong co-movement of OOS rate and defects suggests dimensional root cause
   - Supplier drift signatures show persistent deviation bias / increased OOS

Outputs are designed to be readable by QC engineers and managers.


In [3]:
# ============================================================
# Step 3 — RCA: Explainable root cause analysis for top alerts
# ============================================================

DATA_DIR = "data"

# ----------------------------
# 1) Load the prepared tables
# ----------------------------
daily_kpi = pd.read_csv(os.path.join(DATA_DIR, "daily_kpi.csv"))
alerts = pd.read_csv(os.path.join(DATA_DIR, "alerts_step2.csv"))

parts = pd.read_csv(os.path.join(DATA_DIR, "parts_master.csv"))
suppliers = pd.read_csv(os.path.join(DATA_DIR, "suppliers.csv"))
engineering_changes = pd.read_csv(os.path.join(DATA_DIR, "engineering_changes.csv"))
defects = pd.read_csv(os.path.join(DATA_DIR, "defects.csv"))
dim_measurements = pd.read_csv(os.path.join(DATA_DIR, "dim_measurements.csv"))

# ----------------------------
# 2) Parse dates/timestamps
# ----------------------------
daily_kpi["date"] = pd.to_datetime(daily_kpi["date"])
alerts["date"] = pd.to_datetime(alerts["date"])

engineering_changes["start_date"] = pd.to_datetime(engineering_changes["start_date"])
engineering_changes["end_date"] = pd.to_datetime(engineering_changes["end_date"])

defects["date"] = pd.to_datetime(defects["date"])
dim_measurements["timestamp"] = pd.to_datetime(dim_measurements["timestamp"])
dim_measurements["date"] = dim_measurements["timestamp"].dt.floor("D")

# Normalize ID columns to string for robust joins
for df, col in [(alerts, "supplier_id"), (alerts, "part_id"), (alerts, "plant_line"), (alerts, "shift")]:
    df[col] = df[col].astype(str)

for df, col in [(daily_kpi, "supplier_id"), (daily_kpi, "part_id"), (daily_kpi, "plant_line"), (daily_kpi, "shift")]:
    df[col] = df[col].astype(str)

# ----------------------------
# 3) Choose RCA candidates from alerts
# ----------------------------
# Strategy:
# - Use the most recent alert date
# - Take top N by alert_score across that date
N_CASES = 8

if len(alerts) == 0:
    raise ValueError("No alerts found. Run Step 2 first or lower thresholds.")

latest_alert_date = alerts["date"].max()
alerts_latest = alerts[alerts["date"] == latest_alert_date].copy()
alerts_latest = alerts_latest.sort_values("alert_score", ascending=False).head(N_CASES)

print("RCA will analyze cases from date:", latest_alert_date.date().isoformat())
display(alerts_latest[["date", "supplier_id", "part_id", "plant_line", "shift", "alert_type", "alert_score"]])

# ----------------------------
# 4) Helper functions for RCA evidence
# ----------------------------
GROUP_COLS = ["supplier_id", "part_id", "plant_line", "shift"]

def window_stats(df: pd.DataFrame, end_date: pd.Timestamp, recent_days: int, baseline_days: int):
    """
    Compute recent vs baseline stats ending at end_date (inclusive).
    Baseline excludes the recent window.
    """
    end_date = pd.Timestamp(end_date)

    recent_start = end_date - pd.Timedelta(days=recent_days - 1)
    baseline_end = recent_start - pd.Timedelta(days=1)
    baseline_start = baseline_end - pd.Timedelta(days=baseline_days - 1)

    recent = df[(df["date"] >= recent_start) & (df["date"] <= end_date)]
    baseline = df[(df["date"] >= baseline_start) & (df["date"] <= baseline_end)]

    return recent, baseline, recent_start, baseline_start, baseline_end

def safe_ratio(a, b):
    """Compute a/b with safe handling for zero baseline."""
    if b is None or pd.isna(b) or b == 0:
        return np.nan
    return a / b

def eo_active_on_date(part_id: str, dt: pd.Timestamp, engineering_changes: pd.DataFrame, defects_df: pd.DataFrame, meas_df: pd.DataFrame):
    """
    EO is considered active if:
    - the case has eo_flag already from daily_kpi OR
    - there are any defects/meas records for that part with eo present on that date.
    """
    dt = pd.Timestamp(dt)
    # Check defects: eo_id not null on that date + part match
    d = defects_df[(defects_df["date"] == dt) & (defects_df["part_id"].astype(str) == str(part_id))]
    eo_def = int(d["eo_id"].astype("string").notna().any()) if "eo_id" in d.columns else 0

    # Check measurements: eo_active not null on that date + part match
    m = meas_df[(meas_df["date"] == dt) & (meas_df["part_id"].astype(str) == str(part_id))]
    eo_meas = int(m["eo_active"].astype("string").notna().any()) if "eo_active" in m.columns else 0

    # Check global EO windows (date within any EO window) is less precise without mapping,
    # but we still can flag if date overlaps any EO at all.
    eo_any_window = int(((engineering_changes["start_date"] <= dt) & (engineering_changes["end_date"] >= dt)).any())

    return int((eo_def > 0) or (eo_meas > 0) or (eo_any_window > 0))

# ----------------------------
# 5) Build RCA evidence for each case
# ----------------------------
RECENT_DAYS = 7
BASELINE_DAYS = 30

evidence_rows = []
case_rows = []

# Index daily_kpi by group for faster slicing
daily_kpi_idx = daily_kpi.set_index(GROUP_COLS, drop=False)

for _, case in alerts_latest.iterrows():
    case_date = case["date"]
    key = (case["supplier_id"], case["part_id"], case["plant_line"], case["shift"])

    if key not in daily_kpi_idx.index:
        # If no direct match, skip (should be rare)
        continue

    # Extract timeseries for this case grain
    ts = daily_kpi_idx.loc[key].copy()
    if isinstance(ts, pd.Series):
        ts = ts.to_frame().T
    ts = ts.sort_values("date")

    # Compute recent vs baseline windows
    recent, baseline, recent_start, baseline_start, baseline_end = window_stats(
        ts, end_date=case_date, recent_days=RECENT_DAYS, baseline_days=BASELINE_DAYS
    )

    # Aggregate key signals
    recent_def_rate = recent["defect_rate_per_1000"].mean() if len(recent) else np.nan
    base_def_rate = baseline["defect_rate_per_1000"].mean() if len(baseline) else np.nan
    def_rate_ratio = safe_ratio(recent_def_rate, base_def_rate)

    recent_oos = recent["oos_rate"].mean() if len(recent) else np.nan
    base_oos = baseline["oos_rate"].mean() if len(baseline) else np.nan
    oos_ratio = safe_ratio(recent_oos, base_oos)

    recent_cost = recent["cost_per_1000_units"].mean() if len(recent) else np.nan
    base_cost = baseline["cost_per_1000_units"].mean() if len(baseline) else np.nan
    cost_ratio = safe_ratio(recent_cost, base_cost)

    # Evidence: measurement-defect co-movement (correlation) in recent window
    corr = np.nan
    if len(recent) >= 4:
        corr = recent[["oos_rate", "defect_rate_per_1000"]].corr().iloc[0, 1]

    # EO involvement evidence
    eo_flag_case = int(case.get("eo_flag", 0))
    eo_flag_deep = eo_active_on_date(
        part_id=case["part_id"],
        dt=case_date,
        engineering_changes=engineering_changes,
        defects_df=defects,
        meas_df=dim_measurements
    )
    eo_final = int((eo_flag_case > 0) or (eo_flag_deep > 0))

    # Supplier baseline risk evidence
    sup_row = suppliers[suppliers["supplier_id"].astype(str) == str(case["supplier_id"])]
    supplier_rating = sup_row["quality_rating"].iloc[0] if len(sup_row) else None
    supplier_ppm_base = sup_row["defect_ppm_baseline"].iloc[0] if len(sup_row) else None

    # Part criticality evidence
    part_row = parts[parts["part_id"].astype(str) == str(case["part_id"])]
    part_system = part_row["system"].iloc[0] if len(part_row) else None
    part_ctq = part_row["criticality"].iloc[0] if len(part_row) else None

    # Compute a simple "RCA hypothesis" label using evidence rules
    # This is a heuristic, not a causal proof.
    hypothesis = "Unknown"
    if eo_final == 1 and (def_rate_ratio is not np.nan) and (def_rate_ratio >= 1.5):
        hypothesis = "Change-driven (EO/Model Change) risk"
    if (recent_oos is not np.nan) and (recent_oos >= 0.08) and (corr is not np.nan) and (corr >= 0.4):
        hypothesis = "Dimensional drift likely (OOS ↑ and Defects ↑ together)"
    if supplier_rating in ["B", "C"] and (def_rate_ratio is not np.nan) and (def_rate_ratio >= 1.8):
        hypothesis = "Supplier-driven quality issue likely"

    # Store evidence row
    evidence_rows.append({
        "case_date": case_date.date().isoformat(),
        "supplier_id": case["supplier_id"],
        "part_id": case["part_id"],
        "plant_line": case["plant_line"],
        "shift": case["shift"],
        "alert_type": case["alert_type"],
        "alert_score": float(case["alert_score"]),
        "recent_window_start": recent_start.date().isoformat(),
        "baseline_window_start": baseline_start.date().isoformat(),
        "baseline_window_end": baseline_end.date().isoformat(),
        "recent_defect_rate_per_1000": recent_def_rate,
        "baseline_defect_rate_per_1000": base_def_rate,
        "defect_rate_ratio": def_rate_ratio,
        "recent_oos_rate": recent_oos,
        "baseline_oos_rate": base_oos,
        "oos_ratio": oos_ratio,
        "recent_cost_per_1000": recent_cost,
        "baseline_cost_per_1000": base_cost,
        "cost_ratio": cost_ratio,
        "recent_oos_defect_corr": corr,
        "eo_involved": eo_final,
        "supplier_quality_rating": supplier_rating,
        "supplier_ppm_baseline": supplier_ppm_base,
        "part_system": part_system,
        "part_criticality": part_ctq,
        "rca_hypothesis_v1": hypothesis
    })

    # Create a short case summary row (management-facing)
    case_rows.append({
        "case_date": case_date.date().isoformat(),
        "alert_type": case["alert_type"],
        "supplier_id": case["supplier_id"],
        "part_id": case["part_id"],
        "plant_line": case["plant_line"],
        "shift": case["shift"],
        "rca_hypothesis_v1": hypothesis,
        "key_evidence": (
            f"DefectRate 7d/30d={def_rate_ratio:.2f} | "
            f"OOS 7d={recent_oos:.3f} (base={base_oos:.3f}) | "
            f"Cost 7d/30d={cost_ratio:.2f} | "
            f"EO={eo_final} | "
            f"SupplierRating={supplier_rating} | "
            f"CTQ={part_ctq}"
        )
    })

# Convert to DataFrames
rca_evidence = pd.DataFrame(evidence_rows)
rca_cases = pd.DataFrame(case_rows)

# Sort by alert score descending for readability
rca_cases = rca_cases.sort_values("case_date").sort_values("case_date").reset_index(drop=True)
rca_evidence = rca_evidence.sort_values("alert_score", ascending=False).reset_index(drop=True)

# Save outputs
rca_cases_path = os.path.join(DATA_DIR, "rca_cases_step3.csv")
rca_evidence_path = os.path.join(DATA_DIR, "rca_evidence_step3.csv")

rca_cases.to_csv(rca_cases_path, index=False)
rca_evidence.to_csv(rca_evidence_path, index=False)

print(f"✅ Saved RCA cases: {os.path.abspath(rca_cases_path)}")
print(f"✅ Saved RCA evidence: {os.path.abspath(rca_evidence_path)}")

display(rca_cases)
display(rca_evidence.head(20))

# ----------------------------
# 6) Optional: Identify top drivers across the entire latest alert date
# ----------------------------
# This helps answer: "Which supplier/part contributed the most on the alert day?"
latest_day = latest_alert_date

day_slice = daily_kpi[daily_kpi["date"] == latest_day].copy()
day_slice["supplier_id"] = day_slice["supplier_id"].astype(str)
day_slice["part_id"] = day_slice["part_id"].astype(str)

top_suppliers = (
    day_slice.groupby("supplier_id", as_index=False)
    .agg(defect_n=("defect_n", "sum"), cost_usd=("cost_usd", "sum"), risk=("risk_score_v1", "sum"))
    .sort_values(["risk", "defect_n", "cost_usd"], ascending=False)
    .head(10)
)

top_parts = (
    day_slice.groupby("part_id", as_index=False)
    .agg(defect_n=("defect_n", "sum"), cost_usd=("cost_usd", "sum"), risk=("risk_score_v1", "sum"))
    .sort_values(["risk", "defect_n", "cost_usd"], ascending=False)
    .head(10)
)

print("Top suppliers on latest alert date:")
display(top_suppliers)

print("Top parts on latest alert date:")
display(top_parts)


RCA will analyze cases from date: 2026-01-30


,date,supplier_id,part_id,plant_line,shift,alert_type,alert_score
221,2026-01-30,S008,P0015,ChassisLine-1,B,DEFECT_SPIKE,33.740785
222,2026-01-30,S009,P0030,Final-1,B,DEFECT_SPIKE,32.278929
223,2026-01-30,S008,P0015,ChassisLine-1,B,COST_SPIKE,28.413466
224,2026-01-30,S009,P0030,Final-1,B,COST_SPIKE,25.823143


✅ Saved RCA cases: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\rca_cases_step3.csv
✅ Saved RCA evidence: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\rca_evidence_step3.csv


C:\Users\Hyunwoo Jang\AppData\Local\Temp\ipykernel_29092\3390088783.py:121: PerformanceWarning: indexing past lexsort depth may impact performance.
  if key not in daily_kpi_idx.index:
C:\Users\Hyunwoo Jang\AppData\Local\Temp\ipykernel_29092\3390088783.py:126: PerformanceWarning: indexing past lexsort depth may impact performance.
  ts = daily_kpi_idx.loc[key].copy()
C:\Users\Hyunwoo Jang\AppData\Local\Temp\ipykernel_29092\3390088783.py:121: PerformanceWarning: indexing past lexsort depth may impact performance.
  if key not in daily_kpi_idx.index:
C:\Users\Hyunwoo Jang\AppData\Local\Temp\ipykernel_29092\3390088783.py:126: PerformanceWarning: indexing past lexsort depth may impact performance.
  ts = daily_kpi_idx.loc[key].copy()
C:\Users\Hyunwoo Jang\AppData\Local\Temp\ipykernel_29092\3390088783.py:121: PerformanceWarning: indexing past lexsort depth may impact performance.
  if key not in daily_kpi_idx.index:
C:\Users\Hyunwoo Jang\AppData\Local\Temp\ipykernel_29092\3390088783.py:126:

,case_date,alert_type,supplier_id,part_id,plant_line,shift,rca_hypothesis_v1,key_evidence
0,2026-01-30,DEFECT_SPIKE,S008,P0015,ChassisLine-1,B,Supplier-driven quality issue likely,DefectRate 7d/30d=2.38 | OOS 7d=0.200 (base=0....
1,2026-01-30,DEFECT_SPIKE,S009,P0030,Final-1,B,Unknown,DefectRate 7d/30d=nan | OOS 7d=0.000 (base=0.0...
2,2026-01-30,COST_SPIKE,S008,P0015,ChassisLine-1,B,Supplier-driven quality issue likely,DefectRate 7d/30d=2.38 | OOS 7d=0.200 (base=0....
3,2026-01-30,COST_SPIKE,S009,P0030,Final-1,B,Unknown,DefectRate 7d/30d=nan | OOS 7d=0.000 (base=0.0...


,case_date,supplier_id,part_id,plant_line,shift,alert_type,alert_score,recent_window_start,baseline_window_start,baseline_window_end,...,recent_cost_per_1000,baseline_cost_per_1000,cost_ratio,recent_oos_defect_corr,eo_involved,supplier_quality_rating,supplier_ppm_baseline,part_system,part_criticality,rca_hypothesis_v1
0,2026-01-30,S008,P0015,ChassisLine-1,B,DEFECT_SPIKE,33.740785,2026-01-24,2025-12-25,2026-01-23,...,842.064220,230.93375,3.646345,NaN,0,B,227.92,Chassis,CTQ,Supplier-driven quality issue likely
1,2026-01-30,S009,P0030,Final-1,B,DEFECT_SPIKE,32.278929,2026-01-24,2025-12-25,2026-01-23,...,217.239186,0.00000,NaN,NaN,0,A,152.91,Trim,Non-CTQ,Unknown
2,2026-01-30,S008,P0015,ChassisLine-1,B,COST_SPIKE,28.413466,2026-01-24,2025-12-25,2026-01-23,...,842.064220,230.93375,3.646345,NaN,0,B,227.92,Chassis,CTQ,Supplier-driven quality issue likely
3,2026-01-30,S009,P0030,Final-1,B,COST_SPIKE,25.823143,2026-01-24,2025-12-25,2026-01-23,...,217.239186,0.00000,NaN,NaN,0,A,152.91,Trim,Non-CTQ,Unknown


Top suppliers on latest alert date:


,supplier_id,defect_n,cost_usd,risk
0,S001,0.0,0.00,66.666667
6,S008,1.0,183.57,27.671400
7,S009,1.0,170.75,7.415000
1,S002,0.0,0.00,0.000000
2,S003,0.0,0.00,0.000000
3,S005,0.0,0.00,0.000000
4,S006,0.0,0.00,0.000000
5,S007,0.0,0.00,0.000000
8,S010,0.0,0.00,0.000000
9,S012,0.0,0.00,0.000000


Top parts on latest alert date:


,part_id,defect_n,cost_usd,risk
9,P0018,0.0,0.00,66.666667
8,P0015,1.0,183.57,27.671400
14,P0030,1.0,170.75,7.415000
0,P0002,0.0,0.00,0.000000
1,P0003,0.0,0.00,0.000000
2,P0004,0.0,0.00,0.000000
3,P0006,0.0,0.00,0.000000
4,P0010,0.0,0.00,0.000000
5,P0012,0.0,0.00,0.000000
6,P0013,0.0,0.00,0.000000


## Step 4: Action Tracking & Effect Validation

### Goal
Quantitatively validate whether actions (containment / corrective / preventive / validation tests)
actually improved quality metrics.

This step produces:
1) `action_effectiveness_step4.csv`: pre/post KPI comparison for each action
2) `action_effectiveness_summary_step4.csv`: aggregated view by action type / owner dept

---

### Validation Design (Explainable v1)
For each action with a `start_date`:
- Define a **pre window**: 14 days before start (excluding start day)
- Define a **post window**: 14 days after start (including start day)

We evaluate changes in:
- Defect rate per 1,000
- PPM
- OOS rate
- Cost per 1,000
- Risk score v1

**Interpretation**
- Improvement = metric decreased (for defect rate, ppm, oos, cost, risk)
- We compute absolute delta and percent delta.

---

### Notes / Limitations (v1)
This is an observational before/after comparison.
It is not a fully causal study because:
- Seasonality and production mix can change
- Some actions overlap in time
- Baseline may be noisy for low-volume groups

In Step 5 (dashboard), we will present these results with context (volume, confidence flags).
Later we can upgrade to matched controls / DiD (difference-in-differences).


In [4]:
# ============================================================
# Step 4 — Validation: Pre/Post action effectiveness evaluation
# ============================================================

DATA_DIR = "data"

# ----------------------------
# 1) Load required tables
# ----------------------------
daily_kpi = pd.read_csv(os.path.join(DATA_DIR, "daily_kpi.csv"))
actions = pd.read_csv(os.path.join(DATA_DIR, "actions.csv"))
defects = pd.read_csv(os.path.join(DATA_DIR, "defects.csv"))
dim_measurements = pd.read_csv(os.path.join(DATA_DIR, "dim_measurements.csv"))

# ----------------------------
# 2) Parse dates/timestamps
# ----------------------------
daily_kpi["date"] = pd.to_datetime(daily_kpi["date"])
actions["start_date"] = pd.to_datetime(actions["start_date"], errors="coerce")

# For linking actions to defect/EO scope
defects["date"] = pd.to_datetime(defects["date"])
dim_measurements["timestamp"] = pd.to_datetime(dim_measurements["timestamp"])
dim_measurements["date"] = dim_measurements["timestamp"].dt.floor("D")

# Normalize ID columns to string
for col in ["trigger_type", "trigger_id", "action_type", "owner_dept", "expected_effect_metric"]:
    if col in actions.columns:
        actions[col] = actions[col].astype("string")

# ----------------------------
# 3) Helper: derive "scope keys" for each action
# ----------------------------
# The challenge: actions are triggered by either:
# - a defect_id (trigger_type = "defect") OR
# - an eo_id (trigger_type = "eo")
#
# For validation, we need a consistent grain to compute pre/post metrics.
# We will map each action to a scope:
# - defect-triggered actions: supplier_id + part_id + plant_line + shift (from defects table)
# - eo-triggered actions: all records in daily_kpi where eo_flag == 1 during the action windows
#
# This gives us a meaningful operational scope for effectiveness.

def get_defect_scope(defect_id: str) -> dict:
    """Return scope attributes for a defect-triggered action from defects table."""
    row = defects[defects["defect_id"].astype(str) == str(defect_id)]
    if len(row) == 0:
        return {}
    r = row.iloc[0]
    return {
        "scope_type": "defect_scope",
        "supplier_id": str(r["supplier_id"]),
        "part_id": str(r["part_id"]),
        "plant_line": str(r["plant_line"]),
        "shift": str(r["shift"]),
    }

def get_eo_scope(eo_id: str) -> dict:
    """Return a placeholder scope for EO-triggered actions."""
    return {
        "scope_type": "eo_scope",
        "eo_id": str(eo_id),
    }

# ----------------------------
# 4) Helper: compute pre/post KPI aggregates for a given scope
# ----------------------------
METRICS = [
    "defect_rate_per_1000",
    "ppm",
    "oos_rate",
    "cost_per_1000_units",
    "risk_score_v1",
]

def aggregate_metrics(df: pd.DataFrame) -> dict:
    """Aggregate key metrics using weighted or simple averages."""
    out = {}
    if len(df) == 0:
        for m in METRICS:
            out[m] = np.nan
        out["qty_used_sum"] = 0
        out["meas_n_sum"] = 0
        out["defect_n_sum"] = 0
        out["cost_usd_sum"] = 0
        return out

    # Volume context
    out["qty_used_sum"] = float(df["qty_used"].sum())
    out["meas_n_sum"] = float(df["meas_n"].sum()) if "meas_n" in df.columns else 0.0
    out["defect_n_sum"] = float(df["defect_n"].sum()) if "defect_n" in df.columns else 0.0
    out["cost_usd_sum"] = float(df["cost_usd"].sum()) if "cost_usd" in df.columns else 0.0

    # Defect rate / PPM / cost per 1000 should be recomputed from sums (more stable)
    if out["qty_used_sum"] > 0:
        out["defect_rate_per_1000"] = out["defect_n_sum"] / out["qty_used_sum"] * 1000
        out["ppm"] = out["defect_n_sum"] / out["qty_used_sum"] * 1_000_000
        out["cost_per_1000_units"] = out["cost_usd_sum"] / out["qty_used_sum"] * 1000
    else:
        out["defect_rate_per_1000"] = np.nan
        out["ppm"] = np.nan
        out["cost_per_1000_units"] = np.nan

    # OOS rate: recompute from sums (oos_n / meas_n) if available
    if ("oos_n" in df.columns) and ("meas_n" in df.columns) and (out["meas_n_sum"] > 0):
        out["oos_rate"] = float(df["oos_n"].sum()) / out["meas_n_sum"]
    else:
        # fallback to mean if sums not available
        out["oos_rate"] = float(df["oos_rate"].mean()) if "oos_rate" in df.columns and len(df) else np.nan

    # Risk score: mean is fine as a first-order measure
    out["risk_score_v1"] = float(df["risk_score_v1"].mean()) if "risk_score_v1" in df.columns else np.nan

    return out

def slice_scope(dk: pd.DataFrame, scope: dict) -> pd.DataFrame:
    """Filter daily_kpi to match the action scope."""
    if scope.get("scope_type") == "defect_scope":
        return dk[
            (dk["supplier_id"].astype(str) == scope["supplier_id"]) &
            (dk["part_id"].astype(str) == scope["part_id"]) &
            (dk["plant_line"].astype(str) == scope["plant_line"]) &
            (dk["shift"].astype(str) == scope["shift"])
        ].copy()

    if scope.get("scope_type") == "eo_scope":
        # For EO actions, evaluate on EO-flagged records only (broader scope)
        # This is acceptable for AQIP-style actions where the scope is change-driven.
        return dk[(dk["eo_flag"] == 1)].copy()

    return dk.iloc[0:0].copy()

# ----------------------------
# 5) Pre/Post evaluation loop
# ----------------------------
PRE_DAYS = 14
POST_DAYS = 14

results = []

# Remove actions with missing start_date
actions_valid = actions[actions["start_date"].notna()].copy()

for _, a in actions_valid.iterrows():
    action_id = a["action_id"]
    start_date = pd.Timestamp(a["start_date"]).normalize()

    # Define pre/post windows
    pre_start = start_date - pd.Timedelta(days=PRE_DAYS)
    pre_end = start_date - pd.Timedelta(days=1)

    post_start = start_date
    post_end = start_date + pd.Timedelta(days=POST_DAYS - 1)

    # Build scope
    if a["trigger_type"] == "defect":
        scope = get_defect_scope(a["trigger_id"])
    elif a["trigger_type"] == "eo":
        scope = get_eo_scope(a["trigger_id"])
    else:
        scope = {}

    if not scope:
        # Skip if scope cannot be derived (missing defect reference, etc.)
        continue

    # Slice daily_kpi by scope, then slice by time windows
    dk_scope = slice_scope(daily_kpi, scope)

    pre_df = dk_scope[(dk_scope["date"] >= pre_start) & (dk_scope["date"] <= pre_end)]
    post_df = dk_scope[(dk_scope["date"] >= post_start) & (dk_scope["date"] <= post_end)]

    pre_metrics = aggregate_metrics(pre_df)
    post_metrics = aggregate_metrics(post_df)

    # Compute deltas (post - pre). Negative is "good" for most risk metrics.
    row = {
        "action_id": action_id,
        "trigger_type": str(a["trigger_type"]),
        "trigger_id": str(a["trigger_id"]),
        "action_type": str(a["action_type"]),
        "owner_dept": str(a["owner_dept"]),
        "expected_effect_metric": str(a.get("expected_effect_metric", "")),
        "start_date": start_date.date().isoformat(),
        "pre_window_start": pre_start.date().isoformat(),
        "pre_window_end": pre_end.date().isoformat(),
        "post_window_start": post_start.date().isoformat(),
        "post_window_end": post_end.date().isoformat(),
        "scope_type": scope.get("scope_type"),
        "scope_supplier_id": scope.get("supplier_id", None),
        "scope_part_id": scope.get("part_id", None),
        "scope_plant_line": scope.get("plant_line", None),
        "scope_shift": scope.get("shift", None),
        "scope_eo_id": scope.get("eo_id", None),
    }

    # Attach pre/post metrics and deltas
    for m in METRICS:
        row[f"pre_{m}"] = pre_metrics.get(m, np.nan)
        row[f"post_{m}"] = post_metrics.get(m, np.nan)
        row[f"delta_{m}"] = row[f"post_{m}"] - row[f"pre_{m}"] if pd.notna(row[f"post_{m}"]) and pd.notna(row[f"pre_{m}"]) else np.nan
        row[f"pct_change_{m}"] = (
            row[f"delta_{m}"] / row[f"pre_{m}"]
            if pd.notna(row[f"delta_{m}"]) and pd.notna(row[f"pre_{m}"]) and row[f"pre_{m}"] != 0
            else np.nan
        )

    # Volume context (important to avoid over-interpreting noisy results)
    row["pre_qty_used"] = pre_metrics["qty_used_sum"]
    row["post_qty_used"] = post_metrics["qty_used_sum"]
    row["pre_meas_n"] = pre_metrics["meas_n_sum"]
    row["post_meas_n"] = post_metrics["meas_n_sum"]
    row["pre_defect_n"] = pre_metrics["defect_n_sum"]
    row["post_defect_n"] = post_metrics["defect_n_sum"]
    row["pre_cost_usd"] = pre_metrics["cost_usd_sum"]
    row["post_cost_usd"] = post_metrics["cost_usd_sum"]

    # A simple effectiveness flag (v1): improvement if risk_score and defect rate both decrease
    # (You can customize criteria based on expected_effect_metric later.)
    row["improved_v1"] = int(
        (pd.notna(row["delta_risk_score_v1"]) and row["delta_risk_score_v1"] < 0) and
        (pd.notna(row["delta_defect_rate_per_1000"]) and row["delta_defect_rate_per_1000"] < 0)
    )

    results.append(row)

action_effectiveness = pd.DataFrame(results)

# ----------------------------
# 6) Save action-level effectiveness output
# ----------------------------
out_path = os.path.join(DATA_DIR, "action_effectiveness_step4.csv")
action_effectiveness.to_csv(out_path, index=False)
print(f"✅ Saved action effectiveness table: {os.path.abspath(out_path)}")
print("action_effectiveness shape:", action_effectiveness.shape)

display(action_effectiveness.head(20))

# ----------------------------
# 7) Build summary views (by action_type and owner_dept)
# ----------------------------
if len(action_effectiveness) > 0:
    summary = (
        action_effectiveness
        .groupby(["action_type", "owner_dept"], as_index=False)
        .agg(
            n_actions=("action_id", "count"),
            improved_rate=("improved_v1", "mean"),
            avg_delta_defect_rate=("delta_defect_rate_per_1000", "mean"),
            avg_delta_oos=("delta_oos_rate", "mean"),
            avg_delta_cost=("delta_cost_per_1000_units", "mean"),
            avg_delta_risk=("delta_risk_score_v1", "mean"),
            median_pct_change_defect_rate=("pct_change_defect_rate_per_1000", "median"),
            median_pct_change_oos=("pct_change_oos_rate", "median"),
            median_pct_change_cost=("pct_change_cost_per_1000_units", "median"),
        )
        .sort_values(["improved_rate", "n_actions"], ascending=[False, False])
    )
else:
    summary = pd.DataFrame()

summary_path = os.path.join(DATA_DIR, "action_effectiveness_summary_step4.csv")
summary.to_csv(summary_path, index=False)
print(f"✅ Saved action effectiveness summary: {os.path.abspath(summary_path)}")

display(summary.head(30))

# ----------------------------
# 8) (Optional) Flag low-confidence evaluations
# ----------------------------
# If volumes are too small, pre/post metrics are noisy.
# We'll create a quick diagnostic column.
if len(action_effectiveness) > 0:
    action_effectiveness["low_confidence_flag"] = (
        (action_effectiveness["pre_qty_used"] < 500) |
        (action_effectiveness["post_qty_used"] < 500)
    ).astype(int)

    # Save updated table with confidence flags
    out_path2 = os.path.join(DATA_DIR, "action_effectiveness_step4.csv")
    action_effectiveness.to_csv(out_path2, index=False)
    print("✅ Updated action_effectiveness_step4.csv with low_confidence_flag")


✅ Saved action effectiveness table: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\action_effectiveness_step4.csv
action_effectiveness shape: (131, 46)


,action_id,trigger_type,trigger_id,action_type,owner_dept,expected_effect_metric,start_date,pre_window_start,pre_window_end,post_window_start,...,pct_change_risk_score_v1,pre_qty_used,post_qty_used,pre_meas_n,post_meas_n,pre_defect_n,post_defect_n,pre_cost_usd,post_cost_usd,improved_v1
0,A000000001,defect,D000000558,corrective,ManufacturingEng,oos_drop,2026-01-13,2025-12-30,2026-01-12,2026-01-13,...,-1.000000,793.0,381.0,15.0,5.0,2.0,0.0,265.32,0.00,1
1,A000000002,defect,D000000571,containment,Quality,ppm_drop,2026-01-14,2025-12-31,2026-01-13,2026-01-14,...,-0.242484,2103.0,653.0,109.0,35.0,2.0,0.0,216.71,0.00,1
2,A000000003,defect,D000000493,containment,ManufacturingEng,scrap_cost_drop,2025-12-19,2025-12-05,2025-12-18,2025-12-19,...,-0.547234,275.0,372.0,12.0,16.0,1.0,0.0,103.94,0.00,1
3,A000000004,defect,D000000529,corrective,Production,ppm_drop,2025-12-31,2025-12-17,2025-12-30,2025-12-31,...,-1.000000,273.0,112.0,14.0,17.0,2.0,0.0,271.37,0.00,1
4,A000000005,defect,D000000304,containment,Quality,ppm_drop,2025-10-22,2025-10-08,2025-10-21,2025-10-22,...,0.352877,1670.0,958.0,92.0,32.0,4.0,1.0,438.18,151.13,0
5,A000000006,defect,D000000164,corrective,Production,scrap_cost_drop,2025-08-26,2025-08-12,2025-08-25,2025-08-26,...,NaN,744.0,0.0,9.0,0.0,2.0,0.0,369.00,0.00,0
6,A000000007,defect,D000000352,containment,R&D,scrap_cost_drop,2025-11-06,2025-10-23,2025-11-05,2025-11-06,...,0.865471,1540.0,1287.0,90.0,80.0,1.0,3.0,75.91,353.88,0
7,A000000008,defect,D000000377,containment,R&D,oos_drop,2025-11-13,2025-10-30,2025-11-12,2025-11-13,...,1.479962,775.0,760.0,27.0,28.0,2.0,3.0,336.06,923.52,0
8,A000000009,defect,D000000018,containment,Quality,ppm_drop,2025-07-11,2025-06-27,2025-07-10,2025-07-11,...,-1.000000,894.0,314.0,25.0,21.0,1.0,0.0,69.10,0.00,1
9,A000000010,defect,D000000174,corrective,Quality,ppm_drop,2025-08-28,2025-08-14,2025-08-27,2025-08-28,...,-1.000000,129.0,561.0,8.0,16.0,1.0,0.0,241.24,0.00,1


✅ Saved action effectiveness summary: c:\dev\주식매매 (미국)\WRDS\Projects_repo\Data Analysis\Quality Control\data\action_effectiveness_summary_step4.csv


,action_type,owner_dept,n_actions,improved_rate,avg_delta_defect_rate,avg_delta_oos,avg_delta_cost,avg_delta_risk,median_pct_change_defect_rate,median_pct_change_oos,median_pct_change_cost
0,benchmark,AQIP,1,1.000000,-0.070500,-0.116048,2.228466,-13.544654,-0.046269,-0.514648,0.012428
1,benchmark,Quality,1,1.000000,-0.031757,-0.110975,29.524362,-10.444105,-0.022270,-0.408854,0.178398
14,preventive,Production,6,0.666667,-0.615895,-0.068759,-175.053627,-9.195278,-0.706400,-0.174359,-0.665439
16,preventive,R&D,3,0.666667,-2.930815,-0.171515,-347.465488,-22.899363,-0.853390,-0.777778,-0.801600
2,containment,ManufacturingEng,10,0.600000,-2.031180,-0.008168,-293.013003,-3.836225,-1.000000,-0.297697,-1.000000
7,corrective,ManufacturingEng,10,0.500000,-0.436515,-0.009551,27.322408,-0.277502,-1.000000,-0.615346,-1.000000
13,preventive,ManufacturingEng,6,0.500000,-2.107272,-0.082375,-409.228963,-14.363284,-0.868797,0.134009,-0.927046
15,preventive,Quality,6,0.500000,-2.096514,-0.046767,-297.617311,-8.335937,-1.000000,-0.083333,-1.000000
10,corrective,R&D,2,0.500000,-2.210253,-0.123737,-241.107055,-10.549082,-0.766958,-0.404959,-0.916800
12,countermeasure,Quality,2,0.500000,-0.287829,0.002012,-88.207164,-2.392977,-0.199963,0.018165,-0.377219


✅ Updated action_effectiveness_step4.csv with low_confidence_flag


## Step 5: Dashboard & Reporting (Notebook Decision Support)

### Goal
Deliver a decision-ready QC dashboard **inside the notebook** (no Streamlit, no localhost) that:
1) summarizes current quality status,
2) prioritizes what to investigate,
3) shows likely drivers (RCA),
4) validates whether actions worked (pre/post),
5) supports supplier/line/EO decision-making.

### What you will run in this step
- A notebook dashboard built with **ipywidgets + matplotlib + pandas**
- Interactive filters (date range, supplier, part, line, shift)
- KPI cards + trend charts + top driver tables + latest alerts
- Optional exports (CSV snapshots)

### Data Inputs (from previous steps)
- `data/daily_kpi.csv`
- `data/alerts_step2.csv`
- `data/rca_evidence_step3.csv`
- `data/action_effectiveness_step4.csv`
- `data/action_effectiveness_summary_step4.csv`

### Success Criteria (60-second answers)
- What is the top risk today?
- Which supplier/part/line is driving it?
- Is it likely EO-related or supplier drift?
- What actions worked (and which did not)?
- Where should we focus resources first?


In [5]:
# ============================================================
# Step 5 (Notebook): Load dashboard inputs
# ============================================================

from pathlib import Path

DATA_DIR = Path("data")

required_files = [
    "daily_kpi.csv",
    "alerts_step2.csv",
    "rca_evidence_step3.csv",
    "action_effectiveness_step4.csv",
    "action_effectiveness_summary_step4.csv",
]

missing = [f for f in required_files if not (DATA_DIR / f).exists()]
if missing:
    raise FileNotFoundError(
        "Missing required Step outputs in ./data:\n"
        + "\n".join([f"- {m}" for m in missing])
        + "\n\nRun Step 1~4 to generate these files first."
    )

daily_kpi = pd.read_csv(DATA_DIR / "daily_kpi.csv")
alerts = pd.read_csv(DATA_DIR / "alerts_step2.csv")
rca_evidence = pd.read_csv(DATA_DIR / "rca_evidence_step3.csv")
action_eff = pd.read_csv(DATA_DIR / "action_effectiveness_step4.csv")
action_eff_summary = pd.read_csv(DATA_DIR / "action_effectiveness_summary_step4.csv")

# Parse date columns robustly
daily_kpi["date"] = pd.to_datetime(daily_kpi["date"], errors="coerce")
alerts["date"] = pd.to_datetime(alerts["date"], errors="coerce")

if "case_date" in rca_evidence.columns:
    rca_evidence["case_date"] = pd.to_datetime(rca_evidence["case_date"], errors="coerce")

if "start_date" in action_eff.columns:
    action_eff["start_date"] = pd.to_datetime(action_eff["start_date"], errors="coerce")

print("✅ Loaded Step 5 input tables:")
print(" - daily_kpi:", daily_kpi.shape)
print(" - alerts:", alerts.shape)
print(" - rca_evidence:", rca_evidence.shape)
print(" - action_eff:", action_eff.shape)
print(" - action_eff_summary:", action_eff_summary.shape)

print("\nDate range:", daily_kpi["date"].min().date(), "→", daily_kpi["date"].max().date())


✅ Loaded Step 5 input tables:
 - daily_kpi: (4901, 23)
 - alerts: (225, 24)
 - rca_evidence: (4, 26)
 - action_eff: (131, 47)
 - action_eff_summary: (19, 11)

Date range: 2025-07-01 → 2026-01-31


In [6]:
# ============================================================
# Step 5 (Notebook): Interactive QC Dashboard (No Streamlit)
# ============================================================

import matplotlib.pyplot as plt

from IPython.display import display, clear_output
import ipywidgets as widgets

# ----------------------------
# 0) Helper: safe filtering
# ----------------------------
def apply_filters(df: pd.DataFrame, supplier, part, line, shift) -> pd.DataFrame:
    """Apply categorical filters if columns exist."""
    out = df.copy()
    if supplier != "All" and "supplier_id" in out.columns:
        out = out[out["supplier_id"].astype(str) == str(supplier)]
    if part != "All" and "part_id" in out.columns:
        out = out[out["part_id"].astype(str) == str(part)]
    if line != "All" and "plant_line" in out.columns:
        out = out[out["plant_line"].astype(str) == str(line)]
    if shift != "All" and "shift" in out.columns:
        out = out[out["shift"].astype(str) == str(shift)]
    return out

def compute_rollup(filtered_kpi: pd.DataFrame) -> pd.DataFrame:
    """Compute daily rollup metrics from filtered KPI rows."""
    if len(filtered_kpi) == 0:
        return pd.DataFrame(columns=["date","defect_rate_per_1000","oos_rate","cost_per_1000","risk_mean"])

    roll = (
        filtered_kpi.groupby("date", as_index=False)
        .agg(
            qty_used=("qty_used", "sum"),
            defect_n=("defect_n", "sum"),
            cost_usd=("cost_usd", "sum"),
            meas_n=("meas_n", "sum"),
            oos_n=("oos_n", "sum"),
            risk_mean=("risk_score_v1", "mean") if "risk_score_v1" in filtered_kpi.columns else ("defect_n","mean"),
        )
        .sort_values("date")
    )

    roll["defect_rate_per_1000"] = np.where(roll["qty_used"] > 0, roll["defect_n"] / roll["qty_used"] * 1000, np.nan)
    roll["oos_rate"] = np.where(roll["meas_n"] > 0, roll["oos_n"] / roll["meas_n"], np.nan)
    roll["cost_per_1000"] = np.where(roll["qty_used"] > 0, roll["cost_usd"] / roll["qty_used"] * 1000, np.nan)
    return roll

def kpi_cards(roll: pd.DataFrame):
    """Print KPI cards (text-based) inside notebook output."""
    if len(roll) == 0:
        print("No data for current filters/date range.")
        return

    latest = roll.iloc[-1]
    print("=== KPI Snapshot (Latest Day) ===")
    print(f"Date: {latest['date'].date()}")
    print(f"Defect Rate / 1,000 : {latest['defect_rate_per_1000']:.3f}")
    print(f"OOS Rate           : {latest['oos_rate']:.3%}" if pd.notna(latest["oos_rate"]) else "OOS Rate           : N/A")
    print(f"Cost / 1,000       : ${latest['cost_per_1000']:.2f}")
    if "risk_mean" in latest:
        print(f"Avg Risk Score     : {latest['risk_mean']:.2f}")

# ----------------------------
# 1) Build widget controls
# ----------------------------
min_date = daily_kpi["date"].min()
max_date = daily_kpi["date"].max()

date_picker = widgets.SelectionRangeSlider(
    options=[d.date() for d in pd.date_range(min_date, max_date, freq="D")],
    index=(0, len(pd.date_range(min_date, max_date, freq="D")) - 1),
    description="Date Range",
    layout=widgets.Layout(width="90%")
)

supplier_list = ["All"] + sorted(daily_kpi["supplier_id"].astype(str).unique().tolist()) if "supplier_id" in daily_kpi.columns else ["All"]
part_list = ["All"] + sorted(daily_kpi["part_id"].astype(str).unique().tolist()) if "part_id" in daily_kpi.columns else ["All"]
line_list = ["All"] + sorted(daily_kpi["plant_line"].astype(str).unique().tolist()) if "plant_line" in daily_kpi.columns else ["All"]
shift_list = ["All"] + sorted(daily_kpi["shift"].astype(str).unique().tolist()) if "shift" in daily_kpi.columns else ["All"]

supplier_dd = widgets.Dropdown(options=supplier_list, description="Supplier")
part_dd = widgets.Dropdown(options=part_list, description="Part")
line_dd = widgets.Dropdown(options=line_list, description="Line")
shift_dd = widgets.Dropdown(options=shift_list, description="Shift")

export_btn = widgets.Button(description="Export snapshot CSVs", button_style="success")
out = widgets.Output()

controls_row1 = widgets.HBox([supplier_dd, part_dd, line_dd, shift_dd])
controls_row2 = widgets.HBox([date_picker, export_btn])

display(controls_row1, controls_row2, out)

# ----------------------------
# 2) Dashboard render function
# ----------------------------
def render_dashboard(*_):
    with out:
        clear_output(wait=True)

        start = pd.to_datetime(date_picker.value[0])
        end = pd.to_datetime(date_picker.value[1])

        fk = daily_kpi[(daily_kpi["date"] >= start) & (daily_kpi["date"] <= end)].copy()
        fa = alerts[(alerts["date"] >= start) & (alerts["date"] <= end)].copy()

        fk = apply_filters(fk, supplier_dd.value, part_dd.value, line_dd.value, shift_dd.value)
        fa = apply_filters(fa, supplier_dd.value, part_dd.value, line_dd.value, shift_dd.value)

        roll = compute_rollup(fk)

        # ---- KPI cards
        kpi_cards(roll)

        # ---- Trend charts
        print("\n=== Trends ===")
        if len(roll) >= 2:
            fig = plt.figure(figsize=(10, 3.2))
            plt.plot(roll["date"], roll["defect_rate_per_1000"])
            plt.title("Defect Rate per 1,000 (Daily)")
            plt.xlabel("Date")
            plt.ylabel("Defects / 1,000")
            plt.show()

            fig = plt.figure(figsize=(10, 3.2))
            plt.plot(roll["date"], roll["oos_rate"])
            plt.title("OOS Rate (Daily)")
            plt.xlabel("Date")
            plt.ylabel("OOS Rate")
            plt.show()

            fig = plt.figure(figsize=(10, 3.2))
            plt.plot(roll["date"], roll["cost_per_1000"])
            plt.title("Cost per 1,000 Units (Daily)")
            plt.xlabel("Date")
            plt.ylabel("USD / 1,000")
            plt.show()
        else:
            print("Not enough data points to draw trend charts.")

        # ---- Top drivers (supplier/part)
        print("\n=== Top Drivers (Current Window) ===")
        if len(fk) > 0:
            top_sup = (
                fk.groupby("supplier_id", as_index=False)
                .agg(qty_used=("qty_used","sum"), defect_n=("defect_n","sum"), cost_usd=("cost_usd","sum"),
                     risk=("risk_score_v1","sum") if "risk_score_v1" in fk.columns else ("defect_n","sum"))
                .sort_values(["risk","cost_usd","defect_n"], ascending=False)
                .head(10)
            )
            top_part = (
                fk.groupby("part_id", as_index=False)
                .agg(qty_used=("qty_used","sum"), defect_n=("defect_n","sum"), cost_usd=("cost_usd","sum"),
                     risk=("risk_score_v1","sum") if "risk_score_v1" in fk.columns else ("defect_n","sum"))
                .sort_values(["risk","cost_usd","defect_n"], ascending=False)
                .head(10)
            )
            display(top_sup)
            display(top_part)
        else:
            print("No KPI rows in current filter window.")

        # ---- Latest alerts
        print("\n=== Latest Alerts ===")
        if len(fa) > 0:
            latest_day = fa["date"].max()
            latest_alerts = fa[fa["date"] == latest_day].sort_values("alert_score", ascending=False).head(30)
            display(latest_alerts)
        else:
            print("No alerts found in the selected window/filters.")

# Auto-render on change
for w in [date_picker, supplier_dd, part_dd, line_dd, shift_dd]:
    w.observe(render_dashboard, names="value")

# Initial render
render_dashboard()

# ----------------------------
# 3) Export handler
# ----------------------------
def handle_export(_):
    start = pd.to_datetime(date_picker.value[0])
    end = pd.to_datetime(date_picker.value[1])

    fk = daily_kpi[(daily_kpi["date"] >= start) & (daily_kpi["date"] <= end)].copy()
    fa = alerts[(alerts["date"] >= start) & (alerts["date"] <= end)].copy()

    fk = apply_filters(fk, supplier_dd.value, part_dd.value, line_dd.value, shift_dd.value)
    fa = apply_filters(fa, supplier_dd.value, part_dd.value, line_dd.value, shift_dd.value)

    export_dir = DATA_DIR / "exports"
    export_dir.mkdir(parents=True, exist_ok=True)

    ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    fk.to_csv(export_dir / f"kpi_snapshot_{ts}.csv", index=False)
    fa.to_csv(export_dir / f"alerts_snapshot_{ts}.csv", index=False)

    with out:
        print(f"\n✅ Exported snapshots to: {export_dir.resolve()} (timestamp={ts})")

export_btn.on_click(handle_export)


Output()